In [ ]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
from lxml import html

In [ ]:
#Subject, Subject Code
file_list = pd.Series(['E','GEO','GOV','SSS','US','WH'], index=['Economics','Geography','Government','Social Studies Skills','US History', 'World History'])
keys = ['Subject','Parent Text', 'Parent TID', 'Child Text', 'Child TID', 'Standard Code']

filelist_2= pd.DataFrame(data=[['Economics', 'E'], 
          ['Geography', 'GEO'], 
          ['Government', 'GOV'], 
          ['Social Studies Skills', 'SSS'], 
          ['US History', 'US'], 
          ['World History', 'WH']],
          columns=['Subject', 'Subject_Code'])

In [ ]:
def scrape_standards(standards_list, Subject, Subject_Code):
    filename = r'file:///Users\Melissa\Documents\Udacity\Untitled Folder\High School ' + subject + '.htm'
    file = urlopen(filename)
    soup = BeautifulSoup(file, "lxml")
    standards = soup.ul
    parent = standards.li
    parent_code = 1
    
    while parent != None:
        parent_text = parent.a.contents
        print parent_text
        parent_tid = parent.a['href'][-7:]
        #print parent
        children = parent.ul.findChildren(recursive=False)
        standard_code = subject_code + '.' + str(parent_code)
        child_code = 1
        standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, '', '', standard_code])))    
        #loop through children
        for child in children:
            child_text = child.a.contents
            child_tid = child.a['href'][-7:]
            standard_code = subject_code + '.' + str(parent_code) + '.' + str(child_code)
            standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, child_text, child_tid, standard_code])))    
            child_code += 1
        parent = parent.next_sibling
        print parent
        parent_code += 1
    return(standards_list)

In [ ]:
row = 0
standards_list=[]
while row < 6:
    subject = filelist_2.iloc[row,0]
    subject_code = filelist_2.iloc[row,1]
    scrape_standards(standards_list, subject, subject_code)
    row += 1

In [ ]:
standards_list_df = pd.DataFrame(standards_list)
print standards_list_df

In [ ]:
#output to Excel
output = "Social Studies Taxonomy Export.xlsx"

writer = pd.ExcelWriter(output)
standards_list_df.to_excel(writer, sheet_name='Sheet1')
writer.save()